# This script is used to maniulate the dataset - extract labels and similar

In [1]:
# set app as default directory to address imports
import os
import sys
sys.path.append(os.path.join(os.getcwd(), './../../'))

#activate autoreload to easier test classes
%load_ext autoreload
%autoreload 2

In [2]:
from app.services.sparql_graph import SPARQLGraph
from app.config.enums import Environment
graph = SPARQLGraph(Environment.PROD, False)

Metadata loaded successfully from JSON files.
Initializing SPARQLGraph
Graph loaded with 2056777 triples after 0:02:56.873048


In [3]:
# assign graph to graph
sparql_graph = graph.graph

In [4]:
import numpy as np
import csv
import rdflib

from collections import defaultdict
from rdflib.namespace import Namespace, RDF, RDFS, XSD
WD = Namespace('http://www.wikidata.org/entity/')
WDT = Namespace('http://www.wikidata.org/prop/direct/')

## Create labels directories (No needed to execute to make second part work)

In [16]:
# load the dictionaries
with open('./../too_large_dataset/embeddings/entity_ids.del', 'r') as ifile:
    ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
    id2ent = {v: k for k, v in ent2id.items()}
with open('./../too_large_dataset/embeddings/relation_ids.del', 'r') as ifile:
    rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
    id2rel = {v: k for k, v in rel2id.items()}

In [21]:
ent2lbl = {ent: str(lbl) for ent, lbl in sparql_graph.subject_objects(RDFS.label)}
lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

predicates = set()

for s, p, o in sparql_graph:  # Iterate over all triples in the graph
    predicates.add(p)  # Collect the unique predicates

# Now map each predicate to its label (if available)
rel2lbl = {}

for pred in predicates:
    # Find the label for the predicate, if it exists
    label = sparql_graph.value(subject=pred, predicate=RDFS.label)
    if label:
        rel2lbl[pred] = str(label)
    else:
        # If no label is available, use the URI as the fallback
        rel2lbl[pred] = str(pred)

# Reverse dictionary: labels to relationships
lbl2rel = {lbl: rel for rel, lbl in rel2lbl.items()}

In [24]:
# save everything to json files
import json
with open('./../useful_dataset/graph/ent2lbl.json', 'w') as ofile:
    json.dump({str(k): v for k, v in ent2lbl.items()}, ofile)

with open('./../useful_dataset/graph/lbl2ent.json', 'w') as ofile:
    json.dump({k: str(v) for k, v in lbl2ent.items()}, ofile)

with open('./../useful_dataset/graph/rel2lbl.json', 'w') as ofile:
    json.dump({str(k): v for k, v in rel2lbl.items()}, ofile)

with open('./../useful_dataset/graph/lbl2rel.json', 'w') as ofile:
    json.dump({k: str(v) for k, v in lbl2rel.items()}, ofile)

# Extract Only Movie Labels and Movie-based relations

In [6]:
from collections import defaultdict

# Initialize sets and dictionaries for storing unique items and mappings
unique_movies = set()
unique_relationships = set()
movie2id = {}

# Function to query a movie by its label and return its ID
def get_movie_id_by_label(movie_label):
    # Query the graph to find the ID of a movie based on its label
    query_result = list(graph.graph.subjects(predicate=RDFS.label, object=Literal(movie_label, lang="en")))
    if query_result:
        # Return the first matching ID if found
        return str(query_result[0])
    else:
        # Return None if no match found
        return None

# Assume `graph` is an instance of your `SPARQLGraph` class
# WD['Q11424'] is the identifier for films in the graph (from your 'roots' dictionary)
film_class = WD['Q11424']

# Iterate over all subject-object pairs where the predicate is `P31` (instance of)
for s, o in graph.graph.subject_objects(WDT.P31):
    if o == film_class:
        # Fetch the label for the film entity using the graph function
        film_label = graph.get_lbl_for_ent(str(s))

        if film_label != "Unknown Label":
            # Add the label to the unique movies set
            unique_movies.add(film_label)

            # Map movie label to its ID in movie2id
            movie2id[film_label] = str(s)

            # For each film entity, retrieve all relations (predicates)
            for p, o in graph.graph.predicate_objects(s):
                # Get the label for the predicate (relationship) using the graph function
                rel_label = graph.get_lbl_for_rel(str(p))

                # Add the relationship label if it exists
                if rel_label != "Unknown Label":
                    unique_relationships.add(rel_label)

# Convert the sets to lists for final output
list_of_movies = list(unique_movies)
list_of_relationships = list(unique_relationships)

# # Printing or returning the results as required
# print("Unique Movies:")
# for movie in list_of_movies:
#     print(movie)

# print("\nUnique Relationships:")
# for relationship in list_of_relationships:
#     print(relationship)

# print("\nMovie to ID Mapping:")
# for movie, movie_id in movie2id.items():
#     print(f"{movie}: {movie_id}")


In [7]:
import json

# Convert sets and dictionary to lists for JSON compatibility
list_of_movies = list(unique_movies)
list_of_relationships = list(unique_relationships)

# Define paths for each JSON file
movies_path = './../useful_dataset/graph/unique_movies.json'
relationships_path = './../useful_dataset/graph/unique_relationships.json'
movie2id_path = './../useful_dataset/graph/movie2id.json'

# Save unique movies list to JSON
with open(movies_path, 'w') as ofile:
    json.dump(list_of_movies, ofile)

# Save unique relationships list to JSON
with open(relationships_path, 'w') as ofile:
    json.dump(list_of_relationships, ofile)

# Save movie-to-ID mapping dictionary to JSON
with open(movie2id_path, 'w') as ofile:
    json.dump(movie2id, ofile)

# Random dataset testing

In [9]:
from rdflib.term import Literal

# Specify the entity you want to explore
entity_id = WD['Q47703']  # Replace with the desired entity if different

# Initialize a list to store relationships and connected entity or value labels
relationship_and_entity_pairs = []

# Iterate over all predicate-object pairs where the subject is the specified entity
for p, o in sparql_graph.predicate_objects(entity_id):
    # Get the label for the relationship (predicate)
    rel_label = graph.get_lbl_for_rel(str(p))
    
    # Check if the object is a literal (like a date, number, or other value)
    if isinstance(o, Literal):
        entity_or_value_label = str(o)  # Directly use the literal value (e.g., date, number)
    else:
        # If the object is not a literal, assume it's an entity and get its label
        entity_or_value_label = graph.get_lbl_for_ent(str(o))
    
    # Only add if both the relationship label and entity/value are valid
    if rel_label != "Unknown Label" and entity_or_value_label != "Unknown Label":
        relationship_and_entity_pairs.append((rel_label, entity_or_value_label))

# Printing the results
print(f"Relationships and Entities/Values for entity {entity_id}:")
for rel_label, entity_or_value_label in relationship_and_entity_pairs:
    print(f"{rel_label}: {entity_or_value_label}")

Relationships and Entities/Values for entity http://www.wikidata.org/entity/Q47703:
node label: The Godfather
http://ddis.ch/atai/tag: action
http://ddis.ch/atai/tag: atmospheric
http://ddis.ch/atai/tag: boring
http://ddis.ch/atai/tag: cult
http://ddis.ch/atai/tag: dramatic
http://ddis.ch/atai/tag: entertaining
http://ddis.ch/atai/tag: murder
http://ddis.ch/atai/tag: revenge
http://ddis.ch/atai/tag: violence
node description: 1972 American film directed by Francis Ford Coppola
film editor: Peter Zinner
film editor: William H. Reynolds
movement: New Hollywood
genre: drama
genre: film based on a novel
genre: gangster film
genre: crime film
nominated for: Academy Award for Best Picture
nominated for: Academy Award for Best Director
nominated for: Academy Award for Best Actor
nominated for: Academy Award for Best Supporting Actor
nominated for: Academy Award for Best Writing, Adapted Screenplay
nominated for: Academy Award for Best Original Dramatic Score
nominated for: Academy Award for B